# Camada Bronze

## Importações

In [0]:
%python
import requests
from io import BytesIO
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [0]:
drop table if exists workspace.taxi_trip_bronze.yellow_tripdata_2015_01


In [0]:
%python
# definindo o caminho do volume
volume_path = {
    'yellow_tripdata_2015_01':'/Volumes/workspace/taxi_trip_bronze/landing_zone/yellow_tripdata_2015-01.csv',
    'yellow_tripdata_2016_01':'/Volumes/workspace/taxi_trip_bronze/landing_zone/yellow_tripdata_2016-01.csv',
    'yellow_tripdata_2016_02':'/Volumes/workspace/taxi_trip_bronze/landing_zone/yellow_tripdata_2016-02.csv',
    'yellow_tripdata_2016_03':'/Volumes/workspace/taxi_trip_bronze/landing_zone/yellow_tripdata_2016-03.csv'
}

In [0]:
%python
for table_name, path in volume_path.items():
    print(f"Ingerindo tabela: {table_name}")

    # Leitura do CSV diretamente com Spark
    df_spark = spark.read.option("header", True).csv(path)

    # Padronização técnica (Bronze)
    for col_name in df_spark.columns:
        df_spark = df_spark.withColumn(col_name, col(col_name).cast("string"))
        
    # Escrita na camada Bronze
    (
        df_spark.write
        .mode("overwrite")
        .format("delta")
        .saveAsTable(f"taxi_trip_bronze.{table_name}")
    )

## Conferindo

In [0]:
SELECT * FROM workspace.taxi_trip_bronze.yellow_tripdata_2015_01
LIMIT 5

In [0]:
SELECT * FROM workspace.taxi_trip_bronze.yellow_tripdata_2016_01
LIMIT 5

In [0]:
SELECT * FROM workspace.taxi_trip_bronze.yellow_tripdata_2016_02
LIMIT 5

In [0]:
SELECT * FROM workspace.taxi_trip_bronze.yellow_tripdata_2016_03
LIMIT 5